SPLITS VALIDATION

In [1]:
import pickle
import pandas as pd
import os
import numpy as np

names = ["FB15k", "FB15k237", "WN18", "WN18RR", "YAGO310"]
name_splits = ["Split_0", "Split_1", "Split_2", "Split_3", "Split_4"]
splitted_data = ["training", "testing" ,"validation"]

In [2]:
def check_quality(df):
                #analyze correlations
                c = df.corr(method ='pearson').abs()
                s = c.unstack()
                s = s[s>0.5]
                s = s[s<1]
                for i in range(0,len(s)):
                    print(str(s.axes[0][i]) + " have high correlation")

                #analyze missing values
                if (df.isnull().sum().sum() != 0):
                    print(name + "_" + name_split + "_" + split_data + " have " + str(df.isnull().sum().sum()) + " missing values")

                #check for duplicates
                if (df.duplicated().any()):
                    print(name + "_" + name_split + "_" + split_data + " have " + " duplicated rows")

def check_split(training, testing, validation):

    def check(df1,df2):
        check = pd.merge(df1, df2, on=['h', 'r', 't'], how='left', indicator='Exist')
        check_both = check[check['Exist'] == 'both']
        if (df1.duplicated().any()):
            print(name + "_" + name_split + "_" + split_data + " and " + split_data_0 + " are overlapping")

    check(training,testing)
    check(training,validation)
    check(testing,validation)

In [3]:
for name in names:

    print("Open "+name+"..")

    splits = {
    "Split_0": pd.DataFrame([]),
    "Split_1": pd.DataFrame([]),
    "Split_2": pd.DataFrame([]),
    "Split_3": pd.DataFrame([]),
    "Split_4": pd.DataFrame([])
    }

    for name_split in name_splits:
        kg_path = '../Data_Collection/Datasets_Complete/'+name+'/'+name_split+'/instance.pickle'

        print("Open "+name + "_" + name_split+"..")

        with open(kg_path, 'rb') as f:
            pick = pickle.load(f)

            dataset = pick['dataset']

            data = {
                'dataset': dataset,
                'training': pd.DataFrame(dataset['training'], columns=['h', 'r', 't']),
                'testing': pd.DataFrame(dataset['testing'], columns=['h', 'r', 't']),
                'validation': pd.DataFrame(dataset['validation'], columns=['h', 'r', 't']),
            }

            splits[name_split] = data

            #analyze single split
            for split_data in splitted_data:

                print("Open "+name + "_" + name_split + "_" + split_data + "..")

                df = pd.DataFrame(splits[name_split][split_data])

                #analyze correlations
                c = df.corr(method ='pearson').abs()
                s = c.unstack()
                s = s[s>0.5]
                s = s[s<1]
                for i in range(0,len(s)):
                    print(str(s.axes[0][i]) + " have high correlation")

                df = pd.DataFrame(splits[name_split][split_data])

                #analyze missing values
                if (df.isnull().sum().sum() != 0):
                    print(name + "_" + name_split + "_" + split_data + " have " + str(df.isnull().sum().sum()) + " missing values")

                #check for duplicates
                if (df.duplicated().any()):
                    print(name + "_" + name_split + "_" + split_data + " have " + " duplicated rows")

                #check if the splitting is ok and not overlapping
                for split_data_0 in splitted_data:
                    df1 = df
                    df2 = splits[name_split][split_data_0]
                    check = pd.merge(df1, df2, on=['h', 'r', 't'], how='left', indicator='Exist')
                    check_both = check[check['Exist'] == 'both']
                    if (df1.duplicated().any()):
                        print(name + "_" + name_split + "_" + split_data + " and " + split_data_0 + " are overlapping")

    #percentage of overlapping for different splits
    for split_data in splitted_data:

        overlapping = pd.DataFrame([],columns=name_splits,index=name_splits)

        for name_split in name_splits:

            for name_split_0 in name_splits:

                df1 = splits[name_split][split_data]
                df2 = splits[name_split_0][split_data]
                df_merge = []

                df_merge = pd.merge(df1, df2, on=['h', 'r', 't'], how='inner')
                df1 = df1 + df_merge

                overlap = np.sum(df1.duplicated(keep=False))  # keep=False marks the duplicated row with a True

                overlapping[name_split][name_split_0] = (1-round(overlap / len(df1),4))*100

        print(name + "_" + split_data + " overlapping matrix:")
        print(overlapping.to_markdown())

Open FB15k..
Open FB15k_Split_0..
Open FB15k_Split_0_training..
Open FB15k_Split_0_testing..
Open FB15k_Split_0_validation..
Open FB15k_Split_1..
Open FB15k_Split_1_training..
Open FB15k_Split_1_testing..
Open FB15k_Split_1_validation..
Open FB15k_Split_2..
Open FB15k_Split_2_training..
Open FB15k_Split_2_testing..
Open FB15k_Split_2_validation..
Open FB15k_Split_3..
Open FB15k_Split_3_training..
Open FB15k_Split_3_testing..
Open FB15k_Split_3_validation..
Open FB15k_Split_4..
Open FB15k_Split_4_training..
Open FB15k_Split_4_testing..
Open FB15k_Split_4_validation..
FB15k_training overlapping matrix:
|         |   Split_0 |   Split_1 |   Split_2 |   Split_3 |   Split_4 |
|:--------|----------:|----------:|----------:|----------:|----------:|
| Split_0 |    100    |     80.1  |     80.17 |     80.1  |     80.14 |
| Split_1 |     80.1  |    100    |     80.14 |     80.18 |     80.14 |
| Split_2 |     80.17 |     80.14 |    100    |     80.13 |     80.16 |
| Split_3 |     80.1  |     80.1

ABLATION VALIDATION (CONTAINED INTO BASELINE)

In [12]:
directory = '../Training/TODO/'

for name in names:

    ablations = []

    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)

        if name+"_" in filename:

            print("Open "+ filename+"...")

            if "baseline" in filename:
                with open(f+'/instance.pickle', 'rb') as f_load:
                    pick = pickle.load(f_load)

                    baseline = {
                        'dataset': pick,
                        #'training': pd.DataFrame(pick['training'], columns=['h', 'r', 't']),
                        #'testing': pd.DataFrame(pick['testing'], columns=['h', 'r', 't']),
                        #'validation': pd.DataFrame(pick['validation'], columns=['h', 'r', 't']),
                        #'min_testing': pd.DataFrame(pick['min_testing'], columns=['h', 'r', 't'])

                        'training': set([tuple(x) for x in pick['training']]),
                        'testing': set([tuple(x) for x in pick['testing']]),
                        'validation': set([tuple(x) for x in pick['validation']]),
                        'min_testing': set([tuple(x) for x in pick['min_testing']]),

                    }
            else:
                with open(f+'/instance.pickle', 'rb') as f_load:
                    pick = pickle.load(f_load)

                    abl = {
                        'name': filename,
                        'dataset': pick,
                        'training': set([tuple(x) for x in pick['training']]),
                        'testing': set([tuple(y) for y in pick['testing']]),
                        'validation': set([tuple(z) for z in pick['validation']]),
                        'min_testing': set([tuple(t) for t in pick['min_testing']])
                    }

                    ablations.append(abl)

    training = baseline["training"]
    testing = baseline["testing"]
    validation = baseline["validation"]
    min_test = baseline["min_testing"]

    for a in ablations:

        print("CHECK ABLATION "+ a["name"]+"...")
        if str(a["training"]-training) != "set()":
            print("ERROR IN THE TRAINING ABLATION")
            print(a["training"]-training)

        if str(a["testing"]-testing) != "set()":
            print("ERROR IN THE TESTING ABLATION")
            print(a["testing"]-testing)

        if str(a["validation"]-validation) != "set()":
            print("ERROR IN THE VALIDATION ABLATION")
            print(a["validation"]-validation)

        if str(a["min_testing"]-min_test) != "set()":
            print("ERROR IN THE MIN TESTING ABLATION")
            print(a["min_testing"]-min_test)

Open FB15k_0_ComplEx_harmonic_centrality_0.05...
Open FB15k_0_ComplEx_betweenness_-0.1...
Open FB15k_0_ComplEx_harmonic_centrality_-0.15...
Open FB15k_0_ComplEx_betweenness_-0.15...
Open FB15k_0_ComplEx_harmonic_centrality_-0.2...
Open FB15k_0_ComplEx_baseline_0...
Open FB15k_0_ComplEx_betweenness_0.05...
Open FB15k_0_ComplEx_harmonic_centrality_0.1...
Open FB15k_0_ComplEx_betweenness_0.03...
CHECK ABLATION FB15k_0_ComplEx_harmonic_centrality_0.05...
CHECK ABLATION FB15k_0_ComplEx_betweenness_-0.1...
CHECK ABLATION FB15k_0_ComplEx_harmonic_centrality_-0.15...
CHECK ABLATION FB15k_0_ComplEx_betweenness_-0.15...
CHECK ABLATION FB15k_0_ComplEx_harmonic_centrality_-0.2...
CHECK ABLATION FB15k_0_ComplEx_betweenness_0.05...
CHECK ABLATION FB15k_0_ComplEx_harmonic_centrality_0.1...
CHECK ABLATION FB15k_0_ComplEx_betweenness_0.03...
Open FB15k237_0_ComplEx_harmonic_centrality_-0.1...
Open FB15k237_0_ComplEx_pagerank_0.03...
Open FB15k237_0_ComplEx_pagerank_0.05...
Open FB15k237_0_ComplEx_harmo

OTHER CHECKS ON THE ABLATIONS

In [13]:
directory = '../Training/TODO/'

for name in names:

    ablations = []

    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)

        if name+"_" in filename:

            print("Open "+ filename+"...")

            if "baseline" in filename:
                with open(f+'/instance.pickle', 'rb') as f_load:
                    pick = pickle.load(f_load)

                    baseline = {
                        'dataset': pick,
                        'training': pd.DataFrame(pick['training'], columns=['h', 'r', 't']),
                        'testing': pd.DataFrame(pick['testing'], columns=['h', 'r', 't']),
                        'validation': pd.DataFrame(pick['validation'], columns=['h', 'r', 't']),
                        'min_testing': pd.DataFrame(pick['min_testing'], columns=['h', 'r', 't'])
                    }
            else:
                with open(f+'/instance.pickle', 'rb') as f_load:
                    pick = pickle.load(f_load)

                    abl = {
                        'name': filename,
                        'dataset': pick,
                        'training': pd.DataFrame(pick['training'], columns=['h', 'r', 't']),
                        'testing': pd.DataFrame(pick['testing'], columns=['h', 'r', 't']),
                        'validation': pd.DataFrame(pick['validation'], columns=['h', 'r', 't']),
                        'min_testing': pd.DataFrame(pick['min_testing'], columns=['h', 'r', 't'])
                    }

                    ablations.append(abl)

    training = baseline["training"]
    testing = baseline["testing"]
    validation = baseline["validation"]
    min_test = baseline["min_testing"]

    print("CHECK QUALITY BASELINE")
    check_quality(training)
    check_quality(testing)
    check_quality(validation)
    check_quality(min_test)
    check_split(training,testing,validation)

    print("CHECK QUALITY ABLATIONS")
    for a in ablations:
        print("CHECK ABLATION "+ a["name"]+"...")
        check_quality(a["training"])
        check_quality(a["testing"])
        check_quality(a["validation"])
        check_quality(a["min_testing"])
        check_split(a["training"],a["testing"],a["validation"])

Open FB15k_0_ComplEx_harmonic_centrality_0.05...
Open FB15k_0_ComplEx_betweenness_-0.1...
Open FB15k_0_ComplEx_harmonic_centrality_-0.15...
Open FB15k_0_ComplEx_betweenness_-0.15...
Open FB15k_0_ComplEx_harmonic_centrality_-0.2...
Open FB15k_0_ComplEx_baseline_0...
Open FB15k_0_ComplEx_betweenness_0.05...
Open FB15k_0_ComplEx_harmonic_centrality_0.1...
Open FB15k_0_ComplEx_betweenness_0.03...
CHECK QUALITY BASELINE
CHECK QUALITY ABLATIONS
CHECK ABLATION FB15k_0_ComplEx_harmonic_centrality_0.05...
CHECK ABLATION FB15k_0_ComplEx_betweenness_-0.1...
CHECK ABLATION FB15k_0_ComplEx_harmonic_centrality_-0.15...
CHECK ABLATION FB15k_0_ComplEx_betweenness_-0.15...
CHECK ABLATION FB15k_0_ComplEx_harmonic_centrality_-0.2...
CHECK ABLATION FB15k_0_ComplEx_betweenness_0.05...
CHECK ABLATION FB15k_0_ComplEx_harmonic_centrality_0.1...
CHECK ABLATION FB15k_0_ComplEx_betweenness_0.03...
Open FB15k237_0_ComplEx_harmonic_centrality_-0.1...
Open FB15k237_0_ComplEx_pagerank_0.03...
Open FB15k237_0_ComplEx